# [CONFIG] Configuração básica

## Básicos (numpy, pandas, math, display, locale, time, random, json)

In [477]:
!python -m pip install pandas
!python -m pip install openpyxl
!python -m pip install xlrd
!python -m pip install jupyter
!python -m pip install numpy
# !python -m pip install random
!python -m pip install IPython
!python -m pip install json
# !sudo dpkg-reconfigure locales


from IPython.display import display
import json
import pandas as pd
import math
import numpy as np
import locale
# locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")  # Use "" for auto, or force e.g. to "en_US.UTF-8"

import time
from datetime import datetime, timedelta, date
import random

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


## Visualização (matplotlib, seaborn)

In [478]:
!python -m pip install matplotlib
!python -m pip install seaborn

import matplotlib.pyplot as plt
# import seaborn as sns

## Automação (selenium, schedule, BeautifulSoup)

In [479]:
# !python -m pip install selenium

# from selenium import webdriver
# # from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [480]:
# !python -m pip install schedule
# import schedule

## Web (BeautifulSoup, requests)

In [481]:
!python -m pip install BeautifulSoup
from bs4 import BeautifulSoup

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [482]:
!python -m pip install requests
import requests

## Drive (para ler e escrever arquivos)

In [483]:
from google.colab import drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funções

## Elimina colunas NA

In [486]:
def elimina_colunas_NA(
    base,
    imprime_colunas_vazias = False,
    imprime_colunas_completas = False,
    imprime_colunas_parciais = False,
    remove_da_base = True,
    retorna_parciais = False
    ):

    tamanho_da_base = len(base)

    colunas_vazias = []
    colunas_completas = []
    colunas_parciais = []

    for coluna in base.columns:
        tamanho_da_coluna = len(base[base[coluna].isna()])
        if tamanho_da_coluna == tamanho_da_base:
            # print(coluna)
            colunas_vazias.append(coluna)
        elif tamanho_da_coluna == 0:
            colunas_completas.append(coluna)
        else:
            colunas_parciais.append(coluna)

    if imprime_colunas_vazias == True:
        if len(colunas_vazias) == 0:
            print("Não há colunas NA")
        else:
            print("Colunas vazias: " + str(colunas_vazias))

    if imprime_colunas_completas == True:
        if len(colunas_completas) == 0:
            print("Não há colunas completas")
        else:
            print("Colunas completas: " + str(colunas_completas))

    if imprime_colunas_parciais == True:
        if len(colunas_parciais) == 0:
            print("Não há colunas parciais")
        else:
            print("Colunas parciais: " + str(colunas_parciais))

    if remove_da_base == True:
        base = base.drop(colunas_vazias, axis = 1)

    if retorna_parciais == True:
        return [base, colunas_parciais]
    else:
        return base

## Agrupar cada coluna

In [ ]:
def agrupamento_cada_coluna(
    bd,
    coluna_completa,
    colunas_ignoradas = [],
    ascending = False,
    imprime_tabelas = True,
    ):

    from IPython.display import display

    campos_com_erro = []
    dict_campos = {}

    colunas = bd.columns.drop(coluna_completa)

    if len(colunas_ignoradas) > 0:
        colunas = colunas.drop(colunas_ignoradas)

    for coluna in colunas:
        try:
            temp_coluna = bd.fillna("(vazio)").groupby(coluna).count()[[coluna_completa]].rename(columns = {coluna_completa: "Quantidade"}).sort_values("Quantidade", ascending = ascending)

            if ascending == False:
                temp_coluna["%"] = temp_coluna["Quantidade"]/temp_coluna["Quantidade"].sum()
                temp_coluna["% acumulado"] = temp_coluna["%"].cumsum()

            if imprime_tabelas == True:
                display(temp_coluna)

            dict_campos[coluna] = temp_coluna
            limpa(temp_coluna)

        except:
            campos_com_erro.append(coluna)
            #print("Campo " + coluna + " deu erro =/")

    return [campos_com_erro, dict_campos]

## Análise exploratória básica

In [ ]:
def analise_exploratoria(
    bd,
    imprime_todas_colunas = False,
    imprime_info_colunas = True,
    imprime_colunas_vazias = False,
    imprime_colunas_completas = False,
    imprime_colunas_parciais = False,
    remove_da_base = True,
    retorna_parciais = False,
    detalhar_colunas_parciais = True,
    colunas_ignoradas = []
    ):

    # COMEÇANDO PELAS COLUNAS DISPONÍVEIS E INFO
    if imprime_todas_colunas == True:
        display(bd.columns)

    if imprime_info_colunas == True:
        for i in range(int(np.ceil(len(bd.columns)/20))):
            display(bd.iloc[:, (i*20):min((i+1)*20, len(bd.columns))].info())

    # DETALHAMENTO DE QUAIS COLUNAS SÃO NA OU PARCIAIS
    if retorna_parciais == True:
        [bd_semNA, colunas_parciais] = elimina_colunas_NA(
            bd,
            imprime_colunas_vazias = imprime_colunas_vazias,
            imprime_colunas_completas = imprime_colunas_completas,
            imprime_colunas_parciais = imprime_colunas_parciais,
            remove_da_base = remove_da_base,
            retorna_parciais = retorna_parciais
        )

        if detalhar_colunas_parciais == True:
            for coluna in colunas_parciais:
                print("# " + coluna + ": " + str(len(colunas_parciais[colunas_parciais[coluna].isna()])))
    else:
        colunas_parciais = []

        bd_semNA = elimina_colunas_NA(
            bd,
            imprime_colunas_vazias = imprime_colunas_vazias,
            imprime_colunas_completas = imprime_colunas_completas,
            imprime_colunas_parciais = imprime_colunas_parciais,
            remove_da_base = remove_da_base,
            retorna_parciais = retorna_parciais
        )

    [campos_com_erro, colunas_agrupadas] = agrupamento_cada_coluna(
        bd.reset_index(),
        coluna_completa = bd_semNA.drop(colunas_parciais, axis = 1).reset_index().columns[0],
        colunas_ignoradas = colunas_ignoradas
    )

    if retorna_parciais == True:
        return [bd_semNA, colunas_parciais, colunas_agrupadas, campos_com_erro]
    else:
        return [bd_semNA, colunas_agrupadas, campos_com_erro]

## Completa dígitos (padronizar chaves, CPF, etc)

In [489]:
def completa_digitos(texto, qtd):
    quantidade_zeros = qtd - len(texto)

    contador = quantidade_zeros
    while contador > 0:
        texto = "0" + texto
        contador = contador - 1

    # print(len(texto))

    return texto

## Exemplo de uso
# df["coluna_nova"] = df.apply(lambda x: completa_digitos(x["coluna_referencia"], 2), axis = 1)

## Histograma padrão

In [493]:
def histograma_padrao(
  dados_x,
  titulo = np.nan,
  dados_indice = np.nan,
  tipo = "hist",
  grid = False,
  legend = True,
  linha_media = np.nan,
  qtd_bins = 10,
  espessura = 0.8,
  limite_x_min = np.nan, limite_x_max = np.nan,
  limite_y_min = np.nan, limite_y_max = np.nan,
  ):

  plt.figure(figsize = (20,8))

  if not(pd.isnull(titulo)):
      plt.title(titulo, fontsize = 16)

  if tipo == "hist":
      n, bins, edges = plt.hist(
          dados_x,
          bins = qtd_bins,
          rwidth = espessura
          )
      plt.xticks(bins)

  elif tipo == "bar":
      plt.bar(
          dados_indice,
          dados_x,
          width = espessura
          )



  if not(math.isnan(linha_media)):
      plt.axvline(x=linha_media, color = "black", label='Média')

  if legend == True:
      plt.legend()

  plt.grid(grid)

  if (not(math.isnan(limite_x_max))) & (not(math.isnan(limite_x_min))):
      # print(limite_x_min)
      # print(~math.isnan(limite_x_max))
      plt.xlim(limite_x_min, limite_x_max)

  if (not(math.isnan(limite_y_max))) & (not(math.isnan(limite_y_min))):
      plt.ylim(limite_y_min, limite_y_max)

  plt.show()

## Gráfico com rótulos

In [494]:
def grafico_com_rotulos(
    dados, coluna_x, coluna_y,
    tipo_grafico = "bar", titulo_grafico = "", tamanho_grafico = (20, 8),
    fig = None, ax = None,
    tamanho_fonte = 12, grossura_barra = 1,
    cor_x = "b", rotacao_x = 0,
    formato_eixo_y = "{x:.2f}",
    formato_y = "{:.2f}", rotacao_y = 0, limite_min_y = None, limite_max_y = None,
    xytext_label_y = (0,10),
  ):

  # dados_plot = dados_cliente[dados_cliente["% Faturamento Acumulado"] < 0.40]

  if fig is None:
    fig = plt.figure(figsize = tamanho_grafico)

  plt.clf()
  plt.rcParams.update({'font.size': tamanho_fonte})


  ys = dados[coluna_y]
  xs = np.arange(len(dados[coluna_x]))

  if (tipo_grafico == "bar") | (tipo_grafico == "line"):
    if (tipo_grafico == "bar"):
      plt.bar(
          x = xs,
          height = ys,
          width = grossura_barra,
          label = coluna_y,
      )
    elif (tipo_grafico == "line"):
      plt.plot(
        xs,
        ys,
        cor_x + "o-",
        label = coluna_y,
      )

    # display(cor_x + "o-")

    ax1 = plt.gca()
    if (limite_min_y is not None) & (limite_max_y is not None):
      ax1.set_ylim([min(dados[coluna_y])*limite_min_y, max(dados[coluna_y])*limite_max_y])

    ax1.set_ylabel(coluna_y, color = cor_x)
    ax1.set_xlabel(coluna_x)

    for x,y in zip(xs,ys):
        label = formato_y.format(y)

        plt.annotate(label, # this is the text
                    (x,y), # these are the coordinates to position the label
                    textcoords="offset points", # how to position the text
                    xytext = xytext_label_y, # distance from text to points (x,y)
                    ha='center', # horizontal alignment can be left, right or center
                    color = cor_x,
                    rotation = rotacao_y)


    ax1.xaxis.set_tick_params(rotation = rotacao_x)
    ax1.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter(formato_eixo_y))

    # fig.legend()
    plt.xticks(xs, dados[coluna_x])

  elif tipo_grafico == "barh":
    plt.barh(
        y = xs,
        width = ys,
        height = grossura_barra,
        # label = coluna_y,
    )
    ax1 = plt.gca()
    ax1.set_yticks(xs, dados[coluna_x])

    ax1.set_ylabel(coluna_x)
    ax1.set_xlabel(coluna_y, color = cor_x)
    ax1.invert_yaxis()

    if (limite_min_y is not None) & (limite_max_y is not None):
      ax1.set_xlim([min(dados[coluna_y])*limite_min_y, max(dados[coluna_y])*limite_max_y])

    for x,y in zip(xs,ys):
      label = formato_y.format(y)

      plt.annotate(label, # this is the text
                (y, x), # these are the coordinates to position the label
                textcoords="offset points", # how to position the text
                xytext = (xytext_label_y[1], xytext_label_y[0]), # distance from text to points (x,y)
                ha='center', # horizontal alignment can be left, right or center
                color = cor_x,
                rotation = rotacao_y)

    ax1.xaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter(formato_eixo_y))

  plt.title(titulo_grafico)
  plt.show()

## GridSearch

In [500]:
def treina_modelo_grid(
  x_train,
  y_train,
  modelo,
  param_grid,
  cv = 10,
  scoring = "neg_mean_absolute_error",
  tipo = "grid",
  random_state_seed = 1082141,
  n_iter = 50
):
  from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

  if tipo == "grid":
      grid = GridSearchCV(
          modelo,
          param_grid,
          cv = cv,
          scoring = scoring,
          return_train_score = True,
      )

  elif tipo == "randomized":
      # print("Busca randomizada em " + str(n_iter) + " combinações de parâmetros.")
      grid = RandomizedSearchCV(
          modelo,
          param_grid,
          cv = cv,
          scoring = scoring,
          return_train_score = True,
          random_state = random_state_seed,
          n_iter = n_iter,
      )

  grid.fit(x_train, y_train)

  return grid

### Definimos range de hiperparâmetros
# n_neighbors_range = range(10, 20)
# metric_range = ["cityblock", "cosine", "euclidean", "haversine", "l1", "l2", "manhattan", "nan_euclidean", "minkowski"]
# algorithm_range = ["auto", "ball_tree", "kd_tree", "brute"]

# param_grid = dict(algorithm = algorithm_range)
# # display(param_grid)

# grid_melhor_algorithm = treina_modelo_grid(
#     dados_treino_X,
#     dados_treino_y,
#     KNeighborsClassifier(n_neighbors = 12, metric = "euclidean"),
#     param_grid,
#     cv = 5,
#     scoring = "accuracy",
#     tipo = "grid",
#     random_state_seed = numero_aleatorio,
#     # n_iter = 50
# )

# # print(grid.best_score_)
# # grid.best_estimator_
# tabela_resultados = pd.concat([
#   pd.json_normalize(pd.DataFrame(grid_melhor_algorithm.cv_results_)["params"]),
#   pd.DataFrame(grid_melhor_algorithm.cv_results_)[["rank_test_score", "mean_test_score"]]
# ], axis = 1).set_index("rank_test_score").sort_index()


# plt.figure(figsize = (15,8))
# display(tabela_resultados)
# plt.bar(x = tabela_resultados["algorithm"], height = tabela_resultados["mean_test_score"]);

# Ler da pasta (caso já tenha sido lido dos sites antes)

## Variável global (reler dados)

In [501]:
# var_reler_dados = False
var_reler_dados = True

## Leitura da pasta

In [502]:
if var_reler_dados == False:
  base_jogos = pd.read_excel(
      '/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Base Jogos.xlsx',
      # encoding = 'ISO-8859-1',
      # low_memory=False,
      index_col = 0
    )
  display(base_jogos)

# Brasileirão

## Leitura (site CBF)

In [632]:
if var_reler_dados == True:
  # Make a request to the webpage
  brasileirao_url = 'https://www.cbf.com.br/futebol-brasileiro'
  brasileirao_response = requests.get(brasileirao_url)

  # Create a BeautifulSoup object with the HTML content
  brasileirao_soup = BeautifulSoup(brasileirao_response.content, 'html.parser')

  # soup

### Data e numeração

In [633]:
if var_reler_dados == True:
  # Find all div elements with the class 'clearfix'
  brasileirao_jogos_lista_datas = brasileirao_soup.find_all('span', class_='partida-desc text-1 color-lightgray p-b-15 block uppercase text-center')

  brasileirao_jogos_lista_datas[0]

In [634]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  brasileirao_lista_datas_numeracao = []

  # Iterate over the div elements
  for jogo in brasileirao_jogos_lista_datas:
      # Extract the specific fields from the div and create a dictionary
      jogo_texto = jogo.text.strip()
      # print(jogo.text)

      # Add the dictionary to the data list
      brasileirao_lista_datas_numeracao.append({
          'data_jogo': jogo_texto,
      })

  brasileirao_lista_datas_numeracao

### Local

In [635]:
if var_reler_dados == True:
  # Find all div elements with the class 'clearfix'
  brasileirao_jogos_lista_local = brasileirao_soup.find_all('span', class_='partida-desc text-1 color-lightgray block uppercase text-center')

  brasileirao_jogos_lista_local[0]

In [636]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  brasileirao_lista_local = []

  # Iterate over the div elements
  for jogo in brasileirao_jogos_lista_local:
      # Extract the specific fields from the div and create a dictionary
      local_jogo = jogo.text.strip()
      # print(jogo.text)

      # Add the dictionary to the data list
      brasileirao_lista_local.append({
          'local_jogo': local_jogo,
      })

  brasileirao_lista_local

### Demais detalhes do jogo

In [637]:
if var_reler_dados == True:
  brasileirao_jogos_lista_timeA_timeB_resultado = brasileirao_soup.find_all('div', class_='clearfix')
  brasileirao_jogos_lista_timeA_timeB_resultado[0]

In [638]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  brasileirao_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in brasileirao_jogos_lista_timeA_timeB_resultado:
      # Extract the specific fields from the div and create a dictionary
      link_jogo = jogo.find("a", class_="no-underline").get('href')
      timeA_sigla = jogo.find('div', class_='time pull-left').find('span', class_='time-sigla').text.strip()
      timeA_nome_alt = jogo.find('div', class_='time pull-left').find('img').get("alt")
      timeB_sigla = jogo.find('div', class_='time pull-right').find('span', class_='time-sigla').text.strip()
      timeB_nome_alt = jogo.find('div', class_='time pull-right').find('img').get("alt")
      try:
          resultado = jogo.find('span', class_='bg-blue color-white label-2').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"


      # Add the dictionary to the data list
      brasileirao_lista_timeA_timeB_resultado.append({
          'link_jogo': link_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome_alt,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome_alt,
          'resultado': resultado,
          'fase': np.nan,
          'ida ou volta': "Única",
          'torneio': "Brasileirão",
      })


  brasileirao_lista_timeA_timeB_resultado

In [639]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  brasileirao_base_timeA_timeB_resultado = pd.DataFrame(brasileirao_lista_timeA_timeB_resultado)
  brasileirao_base_datas_numeracao = pd.DataFrame(brasileirao_lista_datas_numeracao)
  brasileirao_base_local = pd.DataFrame(brasileirao_lista_local)

  # # Print or save the JSON and DataFrame
  # print(json_data)
  brasileirao_base_timeA_timeB_resultado_datas_numeracao = brasileirao_base_timeA_timeB_resultado.join(brasileirao_base_datas_numeracao).join(brasileirao_base_local)

  brasileirao_base_timeA_timeB_resultado_datas_numeracao

# Copa do Brasil

## Leitura (site CBF)

### 1ª fase

In [640]:
if var_reler_dados == True:
  # Make a request to the webpage
  copabrasil_url_1fase = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1663'
  copabrasil_response_1fase = requests.get(copabrasil_url_1fase)

  # Create a BeautifulSoup object with the HTML content
  copabrasil_soup_1fase = BeautifulSoup(copabrasil_response_1fase.content, 'html.parser')

  # copabrasil_soup_1fase

In [641]:
if var_reler_dados == True:
  copabrasil_grupos_1fase = copabrasil_soup_1fase.find_all('div', class_='box p-t-25 p-b-15')

  # len(copabrasil_grupos_1fase)
  copabrasil_grupos_1fase[1]

In [642]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_1fase:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg hidden-md').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg hidden-md').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"

      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "1ª Fase",
          'ida ou volta': "Única",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [643]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_1fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_1fase

### 2ª fase

In [644]:
if var_reler_dados == True:
  # Make a request to the webpage
  copabrasil_url_2fase = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1672'
  copabrasil_response_2fase = requests.get(copabrasil_url_2fase)

  # Create a BeautifulSoup object with the HTML content
  copabrasil_soup_2fase = BeautifulSoup(copabrasil_response_2fase.content, 'html.parser')

  # copabrasil_soup_2fase

In [645]:
if var_reler_dados == True:
  copabrasil_grupos_2fase = copabrasil_soup_2fase.find_all('div', class_='box p-t-25 p-b-15')

  # len(copabrasil_grupos_2fase_ida)
  copabrasil_grupos_2fase[1]

In [646]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_2fase:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg hidden-md').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg hidden-md').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"

      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "2ª Fase",
          'ida ou volta': "Única",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [647]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_2fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_2fase

### 3ª fase

In [648]:
if var_reler_dados == True:
  # Make a request to the webpage
  copabrasil_url_3fase = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1679'
  copabrasil_response_3fase = requests.get(copabrasil_url_3fase)

  # Create a BeautifulSoup object with the HTML content
  copabrasil_soup_3fase = BeautifulSoup(copabrasil_response_3fase.content, 'html.parser')

  # soup

#### Ida

In [649]:
if var_reler_dados == True:
  copabrasil_grupos_3fase_ida = copabrasil_soup_3fase.find_all('div', class_='col-md-6 p-b-15 p-t-20 block border-right')

  # len(copabrasil_grupos_oitavas_ida)
  copabrasil_grupos_3fase_ida[1]

In [650]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_3fase_ida:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"

      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "3ª Fase",
          'ida ou volta': "Ida",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [651]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_3fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_3fase

#### Volta

In [652]:
if var_reler_dados == True:
  copabrasil_grupos_3fase_volta = copabrasil_soup_3fase.find_all('div', class_='col-md-6 p-b-15 p-t-20 block')

  # len(copabrasil_grupos_oitavas_volta)
  copabrasil_grupos_3fase_volta[0]

In [653]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_3fase_volta:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"

      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "3ª Fase",
          'ida ou volta': "Volta",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [654]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_3fase = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_3fase

### Oitavas de final

In [655]:
if var_reler_dados == True:
  # Make a request to the webpage
  copabrasil_url_oitavas = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1696'
  copabrasil_response_oitavas = requests.get(copabrasil_url_oitavas)

  # Create a BeautifulSoup object with the HTML content
  copabrasil_soup_oitavas = BeautifulSoup(copabrasil_response_oitavas.content, 'html.parser')

  # soup

#### Ida

In [656]:
if var_reler_dados == True:
  copabrasil_grupos_oitavas_ida = copabrasil_soup_oitavas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block border-right')

  # len(copabrasil_grupos_oitavas_ida)
  copabrasil_grupos_oitavas_ida[0]

In [657]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_oitavas_ida:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"

      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "Oitavas de Final",
          'ida ou volta': "Ida",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [658]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_oitavas_ida = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_oitavas_ida

#### Volta

In [659]:
if var_reler_dados == True:
  copabrasil_grupos_oitavas_volta = copabrasil_soup_oitavas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block')

  # len(copabrasil_grupos_oitavas_volta)
  copabrasil_grupos_oitavas_volta[0]

In [660]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_oitavas_volta:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"

      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "Oitavas de Final",
          'ida ou volta': "Volta",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [661]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_oitavas_volta = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_oitavas_volta

### Quartas de final

In [662]:
if var_reler_dados == True:
  # Make a request to the webpage
  copabrasil_url_quartas = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/copa-brasil-masculino/2023?phase=1703'
  copabrasil_response_quartas = requests.get(copabrasil_url_quartas)

  # Create a BeautifulSoup object with the HTML content
  copabrasil_soup_quartas = BeautifulSoup(copabrasil_response_quartas.content, 'html.parser')

  # soup

#### Ida

In [663]:
if var_reler_dados == True:
  copabrasil_grupos_quartas_ida = copabrasil_soup_quartas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block border-right')

  # len(copabrasil_grupos_oitavas_ida)
  copabrasil_grupos_quartas_ida[0]

In [664]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_quartas_ida:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"


      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "Quartas de Final",
          'ida ou volta': "Ida",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [665]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_quartas_ida = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_quartas_ida

#### Volta

In [666]:
if var_reler_dados == True:
  copabrasil_grupos_quartas_volta = copabrasil_soup_quartas.find_all('div', class_='col-md-6 p-b-15 p-t-20 block')

  # len(copabrasil_grupos_oitavas_volta)
  copabrasil_grupos_quartas_volta[0]

In [667]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  copabrasil_lista_timeA_timeB_resultado = []

  # Iterate over the div elements
  for jogo in copabrasil_grupos_quartas_volta:

      data_jogo = jogo.find("div", class_="text-1 m-b-10 text-center uppercase").text.strip()
      link_jogo = jogo.find("a", class_="no-underline").get("href")
      local_jogo = jogo.find("div", class_="text-1 m-t-10 text-center uppercase").text.strip()
      timeA_sigla = jogo.find('b', class_='text-2 pull-right p-t-15 p-r-10 hidden-lg').text.strip()
      timeA_nome = jogo.find('div', class_='text-2 pull-right p-t-15 p-r-10 hidden-xs hidden-sm hidden-md').text.strip()
      timeB_sigla = jogo.find('b', class_='text-2 pull-left p-t-15 p-l-10 hidden-lg').text.strip()
      timeB_nome = jogo.find('div', class_='text-2 pull-left p-t-15 p-l-20 hidden-xs hidden-sm hidden-md').text.strip()
      try:
          resultado = jogo.find('div', class_='text-3 p-t-15').text.strip()
      except:
          resultado = "Jogo ainda não ocorreu"
      if resultado == "x":
          resultado = "Jogo ainda não ocorreu"

      # Add the dictionary to the data list
      copabrasil_lista_timeA_timeB_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          'local_jogo': local_jogo,
          'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "Quartas de Final",
          'ida ou volta': "Volta",
          'torneio': "Copa do Brasil",
      })


  copabrasil_lista_timeA_timeB_resultado

In [668]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  copabrasil_base_lista_timeA_timeB_resultado_quartas_volta = pd.DataFrame(copabrasil_lista_timeA_timeB_resultado)

  copabrasil_base_lista_timeA_timeB_resultado_quartas_volta

# Tratamento dos dados (site CBF)

## Empilhamento de bases

In [669]:
if var_reler_dados == True:
  base_jogos_cbf = pd.concat([
      brasileirao_base_timeA_timeB_resultado_datas_numeracao,
      copabrasil_base_lista_timeA_timeB_resultado_1fase,
      copabrasil_base_lista_timeA_timeB_resultado_2fase,
      copabrasil_base_lista_timeA_timeB_resultado_3fase,
      copabrasil_base_lista_timeA_timeB_resultado_oitavas_ida,
      copabrasil_base_lista_timeA_timeB_resultado_oitavas_volta,
      copabrasil_base_lista_timeA_timeB_resultado_quartas_ida,
      copabrasil_base_lista_timeA_timeB_resultado_quartas_volta,
  ]).reset_index().drop("index", axis = 1)

  display(base_jogos_cbf.head())
  display(base_jogos_cbf.tail())

  base_jogos_cbf_backup = base_jogos_cbf.copy()
  base_jogos_cbf_tratada = base_jogos_cbf.copy()

,link_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio,data_jogo,local_jogo
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf - MG,FLU,Fluminense - RJ,0 x 3,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Independência - Belo Horizonte - MG\nComo foi ...
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,CUI,Cuiabá Saf - MT,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Allianz Parque - Sao Paulo - SP\nComo foi o jogo
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino - SP,BAH,Bahia - BA,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Nabi Abi Chedid - Braganca Paulista - SP\nComo...
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,GOI,Goiás - GO,2 x 0,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Ligga Arena - Curitiba - PR\nComo foi o jogo
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza - CE,INT,Internacional - RS,1 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Arena Castelão - Fortaleza - CE\nComo foi o jogo


,link_jogo,timeA_sigla,timeA_nome,timeB_sigla,timeB_nome,resultado,fase,ida ou volta,torneio,data_jogo,local_jogo
475,https://www.cbf.com.br/futebol-brasileiro/comp...,FLA,Flamengo - RJ,ATH,Athletico Paranaense - PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,"Qua, 05/07/2023 21:30\r\n - GRU...",Maracanã - Rio de Janeiro - RJ\nDetalhes do jogo
476,https://www.cbf.com.br/futebol-brasileiro/comp...,GRE,Grêmio - RS,BAH,Bahia - BA,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 19:00\r\n -...",Arena do Grêmio - Porto Alegre - RS\nDetalhes ...
477,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,SAO,São Paulo - SP,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qui, 13/07/2023 20:00\r\n -...",Allianz Parque - Sao Paulo - SP\nDetalhes do jogo
478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians - SP,AME,América Fc Saf - MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Neo Química Arena - Sao Paulo - SP\nDetalhes d...
479,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,FLA,Flamengo - RJ,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Ligga Arena - Curitiba - PR\nDetalhes do jogo


## Ajustar nomes colunas

In [670]:
if var_reler_dados == True:
  base_jogos_cbf_tratada.columns = [
      'Link para o jogo',
      'Time mandante (sigla)',
      'Time mandante (nome)',
      'Time visitante (sigla)',
      'Time visitante (nome)',
      'Resultado',
      'Fase do torneio',
      'Ida ou volta',
      'Torneio',
      'Data',
      'Local'
  ]

  display(base_jogos_cbf_tratada.head())
  display(base_jogos_cbf_tratada.tail())

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,Local
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf - MG,FLU,Fluminense - RJ,0 x 3,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Independência - Belo Horizonte - MG\nComo foi ...
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,CUI,Cuiabá Saf - MT,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",Allianz Parque - Sao Paulo - SP\nComo foi o jogo
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino - SP,BAH,Bahia - BA,2 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Nabi Abi Chedid - Braganca Paulista - SP\nComo...
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,GOI,Goiás - GO,2 x 0,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Ligga Arena - Curitiba - PR\nComo foi o jogo
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza - CE,INT,Internacional - RS,1 x 1,NaN,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",Arena Castelão - Fortaleza - CE\nComo foi o jogo


,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,Local
475,https://www.cbf.com.br/futebol-brasileiro/comp...,FLA,Flamengo - RJ,ATH,Athletico Paranaense - PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,"Qua, 05/07/2023 21:30\r\n - GRU...",Maracanã - Rio de Janeiro - RJ\nDetalhes do jogo
476,https://www.cbf.com.br/futebol-brasileiro/comp...,GRE,Grêmio - RS,BAH,Bahia - BA,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 19:00\r\n -...",Arena do Grêmio - Porto Alegre - RS\nDetalhes ...
477,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,SAO,São Paulo - SP,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qui, 13/07/2023 20:00\r\n -...",Allianz Parque - Sao Paulo - SP\nDetalhes do jogo
478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians - SP,AME,América Fc Saf - MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Neo Química Arena - Sao Paulo - SP\nDetalhes d...
479,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,FLA,Flamengo - RJ,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Ligga Arena - Curitiba - PR\nDetalhes do jogo


## Tratar coluna de data e hora

In [671]:
# if var_reler_dados == True:
  # base_jogos_cbf_tratada["Data"].str[4:21].str.strip()[85:100]

In [672]:
if var_reler_dados == True:
  base_jogos_cbf_tratada["Data e Hora"] = pd.to_datetime(base_jogos_cbf_tratada["Data"].str[4:21].str.strip(), format = "%d/%m/%Y %H:%M", errors='coerce')
  base_jogos_cbf_tratada["Ano"] = base_jogos_cbf_tratada["Data e Hora"].dt.year
  base_jogos_cbf_tratada["Mês"] = base_jogos_cbf_tratada["Data e Hora"].dt.month
  base_jogos_cbf_tratada["Dia"] = base_jogos_cbf_tratada["Data e Hora"].dt.day
  base_jogos_cbf_tratada["Hora"] = base_jogos_cbf_tratada["Data e Hora"].dt.hour
  base_jogos_cbf_tratada["Minuto"] = base_jogos_cbf_tratada["Data e Hora"].dt.minute

  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Mês"]).count().iloc[:, 0])
  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Hora"]).count().iloc[:, 0])
  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Minuto"]).count().iloc[:, 0])
  # base_jogos_cbf_tratada[base_jogos_cbf_tratada["Data e Hora"].isna()]

  display(base_jogos_cbf_tratada[(~base_jogos_cbf_tratada["Data e Hora"].isna()) * (base_jogos_cbf_tratada["Torneio"] == "Copa do Brasil")].head())
  display(base_jogos_cbf_tratada[(~base_jogos_cbf_tratada["Data e Hora"].isna()) * (base_jogos_cbf_tratada["Torneio"] == "Copa do Brasil")].tail())

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,Local,Data e Hora,Ano,Mês,Dia,Hora,Minuto
380,https://www.cbf.com.br/futebol-brasileiro/comp...,MAR,Marcílio Dias - SC,CHA,Chapecoense - SC,1 x 0,1ª Fase,Única,Copa do Brasil,"Ter, 21/02/2023 19:00\r\n - GRUPO 1...",Hercílio Luz - Itajai - SC\nComo foi o jogo,2023-02-21 19:00:00,2023.0,2.0,21.0,19.0,0.0
381,https://www.cbf.com.br/futebol-brasileiro/comp...,FAL,Falcon Futebol Clube - SE,VOL,Volta Redonda - RJ,1 x 3,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 16:00\r\n - GRUPO 2...",Batistão - Aracaju - SE\nComo foi o jogo,2023-02-22 16:00:00,2023.0,2.0,22.0,16.0,0.0
382,https://www.cbf.com.br/futebol-brasileiro/comp...,MUT,Mutum Esporte Clube - MT,LON,Londrina - PR,4 x 2,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 17:00\r\n - GRUPO 4...",Valdir Doilho Wons - Nova Mutum - MT\nComo foi...,2023-02-22 17:00:00,2023.0,2.0,22.0,17.0,0.0
383,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Raimundo - RR,CUI,Cuiabá Saf - MT,4 x 3,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",Canarinho - Boa Vista - RR\nComo foi o jogo,2023-02-22 19:00:00,2023.0,2.0,22.0,19.0,0.0
384,https://www.cbf.com.br/futebol-brasileiro/comp...,CAU,Caucaia - CE,TOM,Tombense - MG,0 x 0,1ª Fase,Única,Copa do Brasil,"Qua, 22/02/2023 19:00\r\n - GRUPO 3...",João Ronaldo - Pacajus - CE\nComo foi o jogo,2023-02-22 19:00:00,2023.0,2.0,22.0,19.0,0.0


,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,Local,Data e Hora,Ano,Mês,Dia,Hora,Minuto
475,https://www.cbf.com.br/futebol-brasileiro/comp...,FLA,Flamengo - RJ,ATH,Athletico Paranaense - PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,"Qua, 05/07/2023 21:30\r\n - GRU...",Maracanã - Rio de Janeiro - RJ\nDetalhes do jogo,2023-07-05 21:30:00,2023.0,7.0,5.0,21.0,30.0
476,https://www.cbf.com.br/futebol-brasileiro/comp...,GRE,Grêmio - RS,BAH,Bahia - BA,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 19:00\r\n -...",Arena do Grêmio - Porto Alegre - RS\nDetalhes ...,2023-07-12 19:00:00,2023.0,7.0,12.0,19.0,0.0
477,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras - SP,SAO,São Paulo - SP,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qui, 13/07/2023 20:00\r\n -...",Allianz Parque - Sao Paulo - SP\nDetalhes do jogo,2023-07-13 20:00:00,2023.0,7.0,13.0,20.0,0.0
478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians - SP,AME,América Fc Saf - MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Neo Química Arena - Sao Paulo - SP\nDetalhes d...,2023-07-12 21:30:00,2023.0,7.0,12.0,21.0,30.0
479,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense - PR,FLA,Flamengo - RJ,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",Ligga Arena - Curitiba - PR\nDetalhes do jogo,2023-07-12 21:30:00,2023.0,7.0,12.0,21.0,30.0


## Adicionar rodada do Brasileirão

In [673]:
if var_reler_dados == True:
  base_jogos_cbf_tratada["# jogo"] = base_jogos_cbf_tratada["Data"].str.split("Jogo:").str[1] \
      .str.split(" \r\n                              \r\n                  ").str[0] \
      .str.split(" \r\n              - \r\n          ").str[0] \
      .str.split(" \r\n                      - \r\n              ").str[0] \
      .str.strip().astype("int")

  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Torneio"] == "Brasileirão", "Rodada_temp"] = (((base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Torneio"] == "Brasileirão", "# jogo"]/10).apply(np.ceil)).astype(str).str[:-2])
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Torneio"] == "Brasileirão", "Fase do torneio"] = "Rodada " + \
      base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Torneio"] == "Brasileirão"].apply(lambda x: completa_digitos(x["Rodada_temp"], 2), axis = 1)

  base_jogos_cbf_tratada = base_jogos_cbf_tratada.drop(["Rodada_temp"], axis = 1)

  # base_jogos_cbf_tratada[(base_jogos_cbf_tratada["Torneio"] == "Brasileirão")]["Rodada_temp"]
  # base_jogos_cbf_tratada[["Torneio", "Fase do torneio", "# jogo"]]

  display(pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Fase do torneio"]).count().iloc[:, 0]).head())
  display(pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Fase do torneio"]).count().iloc[:, 0]).tail())

Link para o jogo
Torneio     Fase do torneio                  
Brasileirão Rodada 01                      10
            Rodada 02                      10
            Rodada 03                      10
            Rodada 04                      10
            Rodada 05                      10

Link para o jogo
Torneio        Fase do torneio                   
Copa do Brasil 1ª Fase                         40
               2ª Fase                         20
               3ª Fase                         16
               Oitavas de Final                16
               Quartas de Final                 8

## Separar estado do time

In [674]:
if var_reler_dados == True:
  base_jogos_cbf_tratada["Time mandante (UF)"] = base_jogos_cbf_tratada["Time mandante (nome)"].str[-2:].str.strip()
  base_jogos_cbf_tratada["Time mandante (nome - UF)"] = base_jogos_cbf_tratada["Time mandante (nome)"].str.strip()
  base_jogos_cbf_tratada["Time mandante (nome)"] = base_jogos_cbf_tratada["Time mandante (nome - UF)"].str[:-4].str.strip()

  display(base_jogos_cbf_tratada[["Time mandante (nome - UF)", "Time mandante (nome)", "Time mandante (UF)"]].head())
  display(base_jogos_cbf_tratada[["Time mandante (nome - UF)", "Time mandante (nome)", "Time mandante (UF)"]].tail())

,Time mandante (nome - UF),Time mandante (nome),Time mandante (UF)
0,América Fc Saf - MG,América Fc Saf,MG
1,Palmeiras - SP,Palmeiras,SP
2,Red Bull Bragantino - SP,Red Bull Bragantino,SP
3,Athletico Paranaense - PR,Athletico Paranaense,PR
4,Fortaleza - CE,Fortaleza,CE


,Time mandante (nome - UF),Time mandante (nome),Time mandante (UF)
475,Flamengo - RJ,Flamengo,RJ
476,Grêmio - RS,Grêmio,RS
477,Palmeiras - SP,Palmeiras,SP
478,Corinthians - SP,Corinthians,SP
479,Athletico Paranaense - PR,Athletico Paranaense,PR


In [675]:
# if var_reler_dados == True:
  # base_jogos_cbf_tratada.groupby(["Time mandante (UF)", "Time mandante (nome)"]).count().index
  # base_jogos_cbf_tratada.groupby(["Time mandante (nome)"]).count().index

  # base_jogos_cbf_tratada[base_jogos_cbf_tratada["Time mandante (nome)"] == "Campinense"]

In [676]:
if var_reler_dados == True:
  base_jogos_cbf_tratada["Time visitante (UF)"] = base_jogos_cbf_tratada["Time visitante (nome)"].str[-2:].str.strip()
  base_jogos_cbf_tratada["Time visitante (nome - UF)"] = base_jogos_cbf_tratada["Time visitante (nome)"].str.strip()
  base_jogos_cbf_tratada["Time visitante (nome)"] = base_jogos_cbf_tratada["Time visitante (nome - UF)"].str[:-4].str.strip()

  display(base_jogos_cbf_tratada[["Time visitante (nome - UF)", "Time visitante (nome)", "Time visitante (UF)"]].head())
  display(base_jogos_cbf_tratada[["Time visitante (nome - UF)", "Time visitante (nome)", "Time visitante (UF)"]].tail())

,Time visitante (nome - UF),Time visitante (nome),Time visitante (UF)
0,Fluminense - RJ,Fluminense,RJ
1,Cuiabá Saf - MT,Cuiabá Saf,MT
2,Bahia - BA,Bahia,BA
3,Goiás - GO,Goiás,GO
4,Internacional - RS,Internacional,RS


,Time visitante (nome - UF),Time visitante (nome),Time visitante (UF)
475,Athletico Paranaense - PR,Athletico Paranaense,PR
476,Bahia - BA,Bahia,BA
477,São Paulo - SP,São Paulo,SP
478,América Fc Saf - MG,América Fc Saf,MG
479,Flamengo - RJ,Flamengo,RJ


In [677]:
# if var_reler_dados == True:
  # base_jogos_cbf_tratada.groupby(["Time visitante (UF)", "Time visitante (nome)"]).count().index

## Mais dados sobre local do jogo

In [678]:
if var_reler_dados == True:
  base_jogos_cbf_tratada["Estádio"] = base_jogos_cbf_tratada["Local"].str.replace("\nComo foi o jogo", "").str.replace("\nDetalhes do jogo", "").str.split(" - ").str[0]
  base_jogos_cbf_tratada["Cidade"] = base_jogos_cbf_tratada["Local"].str.replace("\nComo foi o jogo", "").str.replace("\nDetalhes do jogo", "").str.split(" - ").str[1]
  base_jogos_cbf_tratada["UF"] = base_jogos_cbf_tratada["Local"].str.replace("\nComo foi o jogo", "").str.replace("\nDetalhes do jogo", "").str.split(" - ").str[2]

In [679]:
# if var_reler_dados == True:
  # base_jogos.groupby(["UF", "Cidade", "Estádio"]).count().index

## Mais detalhes do resultado

In [680]:
if var_reler_dados == True:
  base_jogos_cbf_tratada["Penaltis_temp"] =  base_jogos_cbf_tratada["Resultado"].str.find("(")
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Penaltis_temp"] < 0, "Houve penaltis?"] = (False)
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Penaltis_temp"] >= 0, "Houve penaltis?"] = (True)

  display(base_jogos_cbf_tratada[["Resultado", "Penaltis_temp", "Houve penaltis?"]].head())
  display(base_jogos_cbf_tratada[["Resultado", "Penaltis_temp", "Houve penaltis?"]].tail())

  base_jogos_cbf_tratada = base_jogos_cbf_tratada.drop("Penaltis_temp", axis = 1)

,Resultado,Penaltis_temp,Houve penaltis?
0,0 x 3,-1,False
1,2 x 1,-1,False
2,2 x 1,-1,False
3,2 x 0,-1,False
4,1 x 1,-1,False


,Resultado,Penaltis_temp,Houve penaltis?
475,Jogo ainda não ocorreu,-1,False
476,Jogo ainda não ocorreu,-1,False
477,Jogo ainda não ocorreu,-1,False
478,Jogo ainda não ocorreu,-1,False
479,Jogo ainda não ocorreu,-1,False


In [681]:
if var_reler_dados == True:
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"] = \
      base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado"].str.split(" x ").str[0].str[-1:].astype(int)
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] == "Jogo ainda não ocorreu", "Gols mandante"] = np.nan

  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"] = \
      base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado"].str.split(" x ").str[1].str[0].astype(int)
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] == "Jogo ainda não ocorreu", "Gols visitante"] = np.nan


  display(base_jogos_cbf_tratada[["Resultado", "Gols mandante", "Gols visitante"]].head())
  display(base_jogos_cbf_tratada[["Resultado", "Gols mandante", "Gols visitante"]].tail())

,Resultado,Gols mandante,Gols visitante
0,0 x 3,0.0,3.0
1,2 x 1,2.0,1.0
2,2 x 1,2.0,1.0
3,2 x 0,2.0,0.0
4,1 x 1,1.0,1.0


,Resultado,Gols mandante,Gols visitante
475,Jogo ainda não ocorreu,NaN,NaN
476,Jogo ainda não ocorreu,NaN,NaN
477,Jogo ainda não ocorreu,NaN,NaN
478,Jogo ainda não ocorreu,NaN,NaN
479,Jogo ainda não ocorreu,NaN,NaN


In [682]:
if var_reler_dados == True:
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = \
      base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"].astype(str).str[0] \
      + " x " + \
      base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"].astype(str).str[0]

  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Resultado"] == "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = "Jogo ainda não ocorreu"

  display(base_jogos_cbf_tratada[["Resultado", "Resultado (sem penaltis)"]].head())
  display(base_jogos_cbf_tratada[base_jogos_cbf_tratada["Houve penaltis?"]][["Resultado", "Resultado (sem penaltis)"]].head())
  display(base_jogos_cbf_tratada[base_jogos_cbf_tratada["Resultado"] == "Jogo ainda não ocorreu"][["Resultado", "Resultado (sem penaltis)"]].head())

,Resultado,Resultado (sem penaltis)
0,0 x 3,0 x 3
1,2 x 1,2 x 1
2,2 x 1,2 x 1
3,2 x 0,2 x 0
4,1 x 1,1 x 1


,Resultado,Resultado (sem penaltis)
428,(6)1 x 1(5),1 x 1
431,(4)1 x 1(2),1 x 1
432,(7)0 x 0(6),0 x 0
433,(4)1 x 1(5),1 x 1
439,(5)0 x 0(6),0 x 0


,Resultado,Resultado (sem penaltis)
120,Jogo ainda não ocorreu,Jogo ainda não ocorreu
121,Jogo ainda não ocorreu,Jogo ainda não ocorreu
122,Jogo ainda não ocorreu,Jogo ainda não ocorreu
123,Jogo ainda não ocorreu,Jogo ainda não ocorreu
124,Jogo ainda não ocorreu,Jogo ainda não ocorreu


In [683]:
if var_reler_dados == True:
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"], "Penaltis mandante"] = \
      base_jogos_cbf_tratada[base_jogos_cbf_tratada["Houve penaltis?"]]["Resultado"].str.split(")").str[0].str.replace("(", "").astype(int)
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"] == False, "Penaltis mandante"] = np.nan

  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"], "Penaltis visitante"] = \
      base_jogos_cbf_tratada[base_jogos_cbf_tratada["Houve penaltis?"]]["Resultado"].str.split("(").str[2].str.replace(")", "").astype(int)
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"] == False, "Penaltis visitante"] = np.nan

  display(base_jogos_cbf_tratada[["Resultado", "Penaltis mandante", "Penaltis visitante"]].head())
  display(base_jogos_cbf_tratada[["Resultado", "Penaltis mandante", "Penaltis visitante"]].tail())

<ipython-input-683-c05706972591>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_jogos_cbf_tratada[base_jogos_cbf_tratada["Houve penaltis?"]]["Resultado"].str.split(")").str[0].str.replace("(", "").astype(int)
<ipython-input-683-c05706972591>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_jogos_cbf_tratada[base_jogos_cbf_tratada["Houve penaltis?"]]["Resultado"].str.split("(").str[2].str.replace(")", "").astype(int)


,Resultado,Penaltis mandante,Penaltis visitante
0,0 x 3,NaN,NaN
1,2 x 1,NaN,NaN
2,2 x 1,NaN,NaN
3,2 x 0,NaN,NaN
4,1 x 1,NaN,NaN


,Resultado,Penaltis mandante,Penaltis visitante
475,Jogo ainda não ocorreu,NaN,NaN
476,Jogo ainda não ocorreu,NaN,NaN
477,Jogo ainda não ocorreu,NaN,NaN
478,Jogo ainda não ocorreu,NaN,NaN
479,Jogo ainda não ocorreu,NaN,NaN


In [684]:
if var_reler_dados == True:
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"], "Resultado dos penaltis"] = \
      base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"], "Penaltis mandante"].astype(str).str[0] \
      + " x " + \
      base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"], "Penaltis visitante"].astype(str).str[0]

  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Houve penaltis?"] == False, "Resultado dos penaltis"] == np.nan

  display(base_jogos_cbf_tratada[base_jogos_cbf_tratada["Houve penaltis?"]][["Resultado", "Resultado dos penaltis"]].head())
  display(base_jogos_cbf_tratada[base_jogos_cbf_tratada["Houve penaltis?"] == False][["Resultado", "Resultado dos penaltis"]].head())

,Resultado,Resultado dos penaltis
428,(6)1 x 1(5),6 x 5
431,(4)1 x 1(2),4 x 2
432,(7)0 x 0(6),7 x 6
433,(4)1 x 1(5),4 x 5
439,(5)0 x 0(6),5 x 6


,Resultado,Resultado dos penaltis
0,0 x 3,NaN
1,2 x 1,NaN
2,2 x 1,NaN
3,2 x 0,NaN
4,1 x 1,NaN


In [685]:
if var_reler_dados == True:
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Gols mandante"] > base_jogos_cbf_tratada["Gols visitante"], "Resultado jogo"] = "Mandante"
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Gols mandante"] < base_jogos_cbf_tratada["Gols visitante"], "Resultado jogo"] = "Visitante"
  base_jogos_cbf_tratada.loc[base_jogos_cbf_tratada["Gols mandante"] == base_jogos_cbf_tratada["Gols visitante"], "Resultado jogo"] = "Empate"

  display(base_jogos_cbf_tratada.head())
  display(base_jogos_cbf_tratada.tail())

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,...,Cidade,UF,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf,FLU,Fluminense,0 x 3,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",...,Belo Horizonte,MG,False,0.0,3.0,0 x 3,NaN,NaN,NaN,Visitante
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,CUI,Cuiabá Saf,2 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",...,Sao Paulo,SP,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino,BAH,Bahia,2 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Braganca Paulista,SP,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense,GOI,Goiás,2 x 0,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Curitiba,PR,False,2.0,0.0,2 x 0,NaN,NaN,NaN,Mandante
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza,INT,Internacional,1 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Fortaleza,CE,False,1.0,1.0,1 x 1,NaN,NaN,NaN,Empate


,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,...,Cidade,UF,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo
475,https://www.cbf.com.br/futebol-brasileiro/comp...,FLA,Flamengo,ATH,Athletico Paranaense,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,"Qua, 05/07/2023 21:30\r\n - GRU...",...,Rio de Janeiro,RJ,False,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN
476,https://www.cbf.com.br/futebol-brasileiro/comp...,GRE,Grêmio,BAH,Bahia,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 19:00\r\n -...",...,Porto Alegre,RS,False,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN
477,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,SAO,São Paulo,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qui, 13/07/2023 20:00\r\n -...",...,Sao Paulo,SP,False,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN
478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians,AME,América Fc Saf,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",...,Sao Paulo,SP,False,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN
479,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense,FLA,Flamengo,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,"Qua, 12/07/2023 21:30\r\n -...",...,Curitiba,PR,False,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN


# Libertadores

## Leitura (site Academia Apostas Brasil)

In [686]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  libertadores_fase_grupos_resultado = []

  for i in range(6):
    # print(i+1)
    rodada = str(i+1)

    libertadores_url_fase_grupos_rodada = 'https://www.academiadasapostasbrasil.com/stats/competition/america-do-sul/241/G1L9QaknaZoPd/2xVym4AL2QRqB/0/'+rodada
    libertadores_response_fase_grupos_rodada = requests.get(libertadores_url_fase_grupos_rodada)

    # Create a BeautifulSoup object with the HTML content
    libertadores_soup_fase_grupos_rodada = BeautifulSoup(libertadores_response_fase_grupos_rodada.content, 'html.parser')

    # libertadores_soup_fase_grupos

    # libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.find_all('tr', class_='even')
    # libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.find_all('td', class_='nowrap')
    libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.\
      find_all('table', class_='competition-rounds competition-half-padding')

    # libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.\
    #   find_all('table', class_='competition-rounds competition-half-padding')[0].\
    #   find_all('a')

    # len(libertadores_fase_grupos_rodada)
    # display(libertadores_fase_grupos_rodada[0].find_all("a")[1].text.strip())
    # display(libertadores_fase_grupos_rodada[0].find_all("a")[6].text.strip())

    # Iterate over the div elements
    for grupo, jogo in enumerate(libertadores_fase_grupos_rodada):
      # print(chr(grupo + 65))

      data_jogo1 = jogo.find_all("td", class_="darker tipsy-active nowrap")[0].get("title")
      link_jogo1 = jogo.find_all("a")[3].get("href")
      timeA_nome1 = jogo.find_all("a")[0].text.strip()
      timeB_nome1 = jogo.find_all("a")[2].text.strip()
      resultado1 = jogo.find_all("a")[1].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta1 = "Ida"
      else:
        ida_ou_volta1 = "Volta"

      # Add the dictionary to the data list
      libertadores_fase_grupos_resultado.append({
          'data_jogo': data_jogo1,
          'link_jogo': link_jogo1,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome1,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome1,
          'resultado': resultado1,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta1,
          'torneio': "Libertadores",
      })


      data_jogo2 = jogo.find_all("td", class_="darker tipsy-active nowrap")[1].get("title")
      link_jogo2 = jogo.find_all("a")[8].get("href")
      timeA_nome2 = jogo.find_all("a")[5].text.strip()
      timeB_nome2 = jogo.find_all("a")[7].text.strip()
      resultado2 = jogo.find_all("a")[6].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta2 = "Ida"
      else:
        ida_ou_volta2 = "Volta"

      libertadores_fase_grupos_resultado.append({
          'data_jogo': data_jogo2,
          'link_jogo': link_jogo2,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome2,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome2,
          'resultado': resultado2,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta2,
          'torneio': "Libertadores",
      })

# libertadores_fase_grupos_resultado

In [687]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  libertadores_base_fase_grupos = pd.DataFrame(libertadores_fase_grupos_resultado)

  display(libertadores_base_fase_grupos.head())
  display(libertadores_base_fase_grupos.tail())

,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
91,29/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Colo Colo,Pereira,0-0,Grupos - Rodada 6 - Grupo F,Volta,Libertadores
92,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Alianza Lima,3-0,Grupos - Rodada 6 - Grupo G,Volta,Libertadores
93,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Libertad,Atlético-MG,1-1,Grupos - Rodada 6 - Grupo G,Volta,Libertadores
94,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Atl. Nacional,Patronato,0-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores
95,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Olimpia,Melgar,4-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores


# Sul-Americana

## Leitura (site Academia Apostas Brasil)

In [688]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  sulamericana_fase_grupos_resultado = []

  for i in range(6):
    # print(i+1)
    rodada = str(i+1)

    sulamericana_url_fase_grupos_rodada = 'https://www.academiadasapostasbrasil.com/stats/competition/america-do-sul/297/kqxeZPb10QroL/gEyAmAr7zQ2po/0/'+rodada
    sulamericana_response_fase_grupos_rodada = requests.get(sulamericana_url_fase_grupos_rodada)

    # Create a BeautifulSoup object with the HTML content
    sulamericana_soup_fase_grupos_rodada = BeautifulSoup(sulamericana_response_fase_grupos_rodada.content, 'html.parser')
    # sulamericana_soup_fase_grupos

    sulamericana_fase_grupos_rodada = sulamericana_soup_fase_grupos_rodada.\
      find_all('table', class_='competition-rounds competition-half-padding')


    # len(sulamericana_fase_grupos_rodada)
    # display(sulamericana_fase_grupos_rodada[0].find_all("a")[1].text.strip())
    # display(sulamericana_fase_grupos_rodada[0].find_all("a")[6].text.strip())

    # Iterate over the div elements
    for grupo, jogo in enumerate(sulamericana_fase_grupos_rodada):
      # print(chr(grupo + 65))

      data_jogo1 = jogo.find_all("td", class_="darker tipsy-active nowrap")[0].get("title")
      link_jogo1 = jogo.find_all("a")[3].get("href")
      timeA_nome1 = jogo.find_all("a")[0].text.strip()
      timeB_nome1 = jogo.find_all("a")[2].text.strip()
      resultado1 = jogo.find_all("a")[1].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta1 = "Ida"
      else:
        ida_ou_volta1 = "Volta"

      # Add the dictionary to the data list
      sulamericana_fase_grupos_resultado.append({
          'data_jogo': data_jogo1,
          'link_jogo': link_jogo1,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome1,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome1,
          'resultado': resultado1,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta1,
          'torneio': "Sul-Americana",
      })


      data_jogo2 = jogo.find_all("td", class_="darker tipsy-active nowrap")[1].get("title")
      link_jogo2 = jogo.find_all("a")[8].get("href")
      timeA_nome2 = jogo.find_all("a")[5].text.strip()
      timeB_nome2 = jogo.find_all("a")[7].text.strip()
      resultado2 = jogo.find_all("a")[6].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta2 = "Ida"
      else:
        ida_ou_volta2 = "Volta"

      sulamericana_fase_grupos_resultado.append({
          'data_jogo': data_jogo2,
          'link_jogo': link_jogo2,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome2,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome2,
          'resultado': resultado2,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta2,
          'torneio': "Sul-Americana",
      })

# sulamericana_fase_grupos_resultado

In [689]:
if var_reler_dados == True:
  # # Convert the data to a pandas DataFrame
  sulamericana_base_fase_grupos = pd.DataFrame(sulamericana_fase_grupos_resultado)

  display(sulamericana_base_fase_grupos.head())
  display(sulamericana_base_fase_grupos.tail())

,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,César Vallejo,LDU Quito,1-2,Grupos - Rodada 1 - Grupo A,Ida,Sul-Americana
1,06/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Magallanes,Botafogo,2-2,Grupos - Rodada 1 - Grupo A,Ida,Sul-Americana
2,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Danubio,Emelec,2-0,Grupos - Rodada 1 - Grupo B,Ida,Sul-Americana
3,06/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Huracán,Guaraní,4-1,Grupos - Rodada 1 - Grupo B,Ida,Sul-Americana
4,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,O. Petrolero,Estudiantes,0-1,Grupos - Rodada 1 - Grupo C,Ida,Sul-Americana


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
91,29/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Def y Justicia,Millonarios,3-1,Grupos - Rodada 6 - Grupo F,Volta,Sul-Americana
92,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Universitario,Gimn La Plata,1-0,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
93,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Santa Fe,Goiás,1-2,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
94,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,San Lorenzo,Estudiantes M.,4-1,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana
95,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Palestino,Fortaleza,1-2,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana


# Tratamento dos dados (site Academia Apostas Brasil)

## Empilhamento de bases

In [690]:
if var_reler_dados == True:
  base_jogos_academia_apostas = pd.concat([
      libertadores_base_fase_grupos,
      sulamericana_base_fase_grupos,
  ]).reset_index().drop("index", axis = 1)

  display(base_jogos_academia_apostas.head())
  display(base_jogos_academia_apostas.tail())

,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
187,29/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Def y Justicia,Millonarios,3-1,Grupos - Rodada 6 - Grupo F,Volta,Sul-Americana
188,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Universitario,Gimn La Plata,1-0,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
189,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Santa Fe,Goiás,1-2,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
190,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,San Lorenzo,Estudiantes M.,4-1,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana
191,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Palestino,Fortaleza,1-2,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana


In [691]:
base_jogos_academia_apostas_backup = base_jogos_academia_apostas.copy()
base_jogos_academia_apostas_tratada = base_jogos_academia_apostas.copy()

## Ajustar nomes colunas

In [692]:
base_jogos_academia_apostas.columns

Index(['data_jogo', 'link_jogo', 'timeA_nome', 'timeB_nome', 'resultado',
       'fase', 'ida ou volta', 'torneio'],
      dtype='object')

In [693]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.columns = [
      'Data',
      'Link para o jogo',
      # 'Time mandante (sigla)',
      'Time mandante (nome)',
      # 'Time visitante (sigla)',
      'Time visitante (nome)',
      'Resultado',
      'Fase do torneio',
      'Ida ou volta',
      'Torneio',
      # 'Local'
  ]

  display(base_jogos_academia_apostas_tratada.head())
  display(base_jogos_academia_apostas_tratada.tail())

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores


,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio
187,29/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Def y Justicia,Millonarios,3-1,Grupos - Rodada 6 - Grupo F,Volta,Sul-Americana
188,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Universitario,Gimn La Plata,1-0,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
189,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Santa Fe,Goiás,1-2,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
190,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,San Lorenzo,Estudiantes M.,4-1,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana
191,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Palestino,Fortaleza,1-2,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana


## Tratar coluna de data e hora

In [694]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada["Data e Hora"] = pd.to_datetime(base_jogos_academia_apostas_tratada["Data"].str[:-1].str.strip(), format = "%d/%m/%Y %H:%M", errors='coerce')
  base_jogos_academia_apostas_tratada["Ano"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.year
  base_jogos_academia_apostas_tratada["Mês"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.month
  base_jogos_academia_apostas_tratada["Dia"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.day
  base_jogos_academia_apostas_tratada["Hora"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.hour
  base_jogos_academia_apostas_tratada["Minuto"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.minute

  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Mês"]).count().iloc[:, 0])
  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Hora"]).count().iloc[:, 0])
  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Minuto"]).count().iloc[:, 0])
  # base_jogos_cbf_tratada[base_jogos_cbf_tratada["Data e Hora"].isna()]

  display(base_jogos_academia_apostas_tratada[(~base_jogos_academia_apostas_tratada["Data e Hora"].isna()) * (base_jogos_academia_apostas_tratada["Torneio"] == "Libertadores")].head())
  display(base_jogos_academia_apostas_tratada[(~base_jogos_academia_apostas_tratada["Data e Hora"].isna()) * (base_jogos_academia_apostas_tratada["Torneio"] == "Libertadores")].tail())

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,Mês,Dia,Hora,Minuto
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 19:00:00,2023,4,5,19,0
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 21:00:00,2023,4,5,21,0
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 21:00:00,2023,4,4,21,0
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 23:00:00,2023,4,4,23,0
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores,2023-04-05 21:00:00,2023,4,5,21,0


,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,Mês,Dia,Hora,Minuto
91,29/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Colo Colo,Pereira,0-0,Grupos - Rodada 6 - Grupo F,Volta,Libertadores,2023-06-29 19:00:00,2023,6,29,19,0
92,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Alianza Lima,3-0,Grupos - Rodada 6 - Grupo G,Volta,Libertadores,2023-06-27 19:00:00,2023,6,27,19,0
93,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Libertad,Atlético-MG,1-1,Grupos - Rodada 6 - Grupo G,Volta,Libertadores,2023-06-27 19:00:00,2023,6,27,19,0
94,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Atl. Nacional,Patronato,0-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores,2023-06-27 21:00:00,2023,6,27,21,0
95,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Olimpia,Melgar,4-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores,2023-06-27 21:00:00,2023,6,27,21,0


## Mais detalhes do resultado

In [695]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada["Penaltis_temp"] =  base_jogos_academia_apostas_tratada["Resultado"].str.find("(")
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Penaltis_temp"] < 0, "Houve penaltis?"] = (False)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Penaltis_temp"] >= 0, "Houve penaltis?"] = (True)

  display(base_jogos_academia_apostas_tratada[["Resultado", "Penaltis_temp", "Houve penaltis?"]].head())
  display(base_jogos_academia_apostas_tratada[["Resultado", "Penaltis_temp", "Houve penaltis?"]].tail())

  base_jogos_academia_apostas_tratada = base_jogos_academia_apostas_tratada.drop("Penaltis_temp", axis = 1)

,Resultado,Penaltis_temp,Houve penaltis?
0,2-1,-1,False
1,0-2,-1,False
2,1-1,-1,False
3,1-2,-1,False
4,2-1,-1,False


,Resultado,Penaltis_temp,Houve penaltis?
187,3-1,-1,False
188,1-0,-1,False
189,1-2,-1,False
190,4-1,-1,False
191,1-2,-1,False


In [696]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"] = \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado"].str.split("-").str[0].str[-1:].astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu", "Gols mandante"] = np.nan

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"] = \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado"].str.split("-").str[1].str[0].astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu", "Gols visitante"] = np.nan


  display(base_jogos_academia_apostas_tratada[["Resultado", "Gols mandante", "Gols visitante"]].head())
  display(base_jogos_academia_apostas_tratada[["Resultado", "Gols mandante", "Gols visitante"]].tail())

<ipython-input-696-33e2aa2024a2>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"] = \
<ipython-input-696-33e2aa2024a2>:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"] = \


,Resultado,Gols mandante,Gols visitante
0,2-1,2.0,1.0
1,0-2,0.0,2.0
2,1-1,1.0,1.0
3,1-2,1.0,2.0
4,2-1,2.0,1.0


,Resultado,Gols mandante,Gols visitante
187,3-1,3.0,1.0
188,1-0,1.0,0.0
189,1-2,1.0,2.0
190,4-1,4.0,1.0
191,1-2,1.0,2.0


In [697]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"].astype(str).str[0] \
      + " x " + \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"].astype(str).str[0]

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = "Jogo ainda não ocorreu"

  display(base_jogos_academia_apostas_tratada[["Resultado", "Resultado (sem penaltis)"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]][["Resultado", "Resultado (sem penaltis)"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu"][["Resultado", "Resultado (sem penaltis)"]].head())

,Resultado,Resultado (sem penaltis)
0,2-1,2 x 1
1,0-2,0 x 2
2,1-1,1 x 1
3,1-2,1 x 2
4,2-1,2 x 1


,Resultado,Resultado (sem penaltis)


,Resultado,Resultado (sem penaltis)


In [698]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis mandante"] = \
      base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]]["Resultado"].str.split(")").str[0].str.replace("(", "").astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False, "Penaltis mandante"] = np.nan

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis visitante"] = \
      base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]]["Resultado"].str.split("(").str[2].str.replace(")", "").astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False, "Penaltis visitante"] = np.nan

  display(base_jogos_academia_apostas_tratada[["Resultado", "Penaltis mandante", "Penaltis visitante"]].head())
  display(base_jogos_academia_apostas_tratada[["Resultado", "Penaltis mandante", "Penaltis visitante"]].tail())

<ipython-input-698-25fbbe8515c6>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]]["Resultado"].str.split(")").str[0].str.replace("(", "").astype(int)
<ipython-input-698-25fbbe8515c6>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]]["Resultado"].str.split("(").str[2].str.replace(")", "").astype(int)


,Resultado,Penaltis mandante,Penaltis visitante
0,2-1,NaN,NaN
1,0-2,NaN,NaN
2,1-1,NaN,NaN
3,1-2,NaN,NaN
4,2-1,NaN,NaN


,Resultado,Penaltis mandante,Penaltis visitante
187,3-1,NaN,NaN
188,1-0,NaN,NaN
189,1-2,NaN,NaN
190,4-1,NaN,NaN
191,1-2,NaN,NaN


In [699]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Resultado dos penaltis"] = \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis mandante"].astype(str).str[0] \
      + " x " + \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis visitante"].astype(str).str[0]

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False, "Resultado dos penaltis"] == np.nan

  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]][["Resultado", "Resultado dos penaltis"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False][["Resultado", "Resultado dos penaltis"]].head())

,Resultado,Resultado dos penaltis


,Resultado,Resultado dos penaltis
0,2-1,NaN
1,0-2,NaN
2,1-1,NaN
3,1-2,NaN
4,2-1,NaN


In [700]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Gols mandante"] > base_jogos_academia_apostas_tratada["Gols visitante"], "Resultado jogo"] = "Mandante"
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Gols mandante"] < base_jogos_academia_apostas_tratada["Gols visitante"], "Resultado jogo"] = "Visitante"
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Gols mandante"] == base_jogos_academia_apostas_tratada["Gols visitante"], "Resultado jogo"] = "Empate"

  display(base_jogos_academia_apostas_tratada.head())
  display(base_jogos_academia_apostas_tratada.tail())

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Hora,Minuto,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 19:00:00,2023,...,19,0,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 21:00:00,2023,...,21,0,False,0.0,2.0,0 x 2,NaN,NaN,NaN,Visitante
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 21:00:00,2023,...,21,0,False,1.0,1.0,1 x 1,NaN,NaN,NaN,Empate
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 23:00:00,2023,...,23,0,False,1.0,2.0,1 x 2,NaN,NaN,NaN,Visitante
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores,2023-04-05 21:00:00,2023,...,21,0,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante


,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Hora,Minuto,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo
187,29/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Def y Justicia,Millonarios,3-1,Grupos - Rodada 6 - Grupo F,Volta,Sul-Americana,2023-06-29 21:00:00,2023,...,21,0,False,3.0,1.0,3 x 1,NaN,NaN,NaN,Mandante
188,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Universitario,Gimn La Plata,1-0,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana,2023-06-28 23:00:00,2023,...,23,0,False,1.0,0.0,1 x 0,NaN,NaN,NaN,Mandante
189,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Santa Fe,Goiás,1-2,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana,2023-06-28 23:00:00,2023,...,23,0,False,1.0,2.0,1 x 2,NaN,NaN,NaN,Visitante
190,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,San Lorenzo,Estudiantes M.,4-1,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana,2023-06-27 19:00:00,2023,...,19,0,False,4.0,1.0,4 x 1,NaN,NaN,NaN,Mandante
191,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Palestino,Fortaleza,1-2,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana,2023-06-27 19:00:00,2023,...,19,0,False,1.0,2.0,1 x 2,NaN,NaN,NaN,Visitante


# Consolidação dos dados lidos das fontes e tratados

In [701]:
base_jogos_tratada = pd.concat([
    base_jogos_cbf_tratada,
    base_jogos_academia_apostas_tratada,
])

display(base_jogos_tratada.head())
display(base_jogos_tratada.tail())

<ipython-input-701-1fc62bea9b62>:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  base_jogos_tratada = pd.concat([


,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,...,Cidade,UF,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf,FLU,Fluminense,0 x 3,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",...,Belo Horizonte,MG,False,0.0,3.0,0 x 3,NaN,NaN,NaN,Visitante
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,CUI,Cuiabá Saf,2 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",...,Sao Paulo,SP,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino,BAH,Bahia,2 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Braganca Paulista,SP,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense,GOI,Goiás,2 x 0,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Curitiba,PR,False,2.0,0.0,2 x 0,NaN,NaN,NaN,Mandante
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza,INT,Internacional,1 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Fortaleza,CE,False,1.0,1.0,1 x 1,NaN,NaN,NaN,Empate


,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,...,Cidade,UF,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo
187,https://www.academiadasapostasbrasil.com/stats...,NaN,Def y Justicia,NaN,Millonarios,3-1,Grupos - Rodada 6 - Grupo F,Volta,Sul-Americana,29/06/2023 21:00h,...,NaN,NaN,False,3.0,1.0,3 x 1,NaN,NaN,NaN,Mandante
188,https://www.academiadasapostasbrasil.com/stats...,NaN,Universitario,NaN,Gimn La Plata,1-0,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana,28/06/2023 23:00h,...,NaN,NaN,False,1.0,0.0,1 x 0,NaN,NaN,NaN,Mandante
189,https://www.academiadasapostasbrasil.com/stats...,NaN,Santa Fe,NaN,Goiás,1-2,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana,28/06/2023 23:00h,...,NaN,NaN,False,1.0,2.0,1 x 2,NaN,NaN,NaN,Visitante
190,https://www.academiadasapostasbrasil.com/stats...,NaN,San Lorenzo,NaN,Estudiantes M.,4-1,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana,27/06/2023 19:00h,...,NaN,NaN,False,4.0,1.0,4 x 1,NaN,NaN,NaN,Mandante
191,https://www.academiadasapostasbrasil.com/stats...,NaN,Palestino,NaN,Fortaleza,1-2,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana,27/06/2023 19:00h,...,NaN,NaN,False,1.0,2.0,1 x 2,NaN,NaN,NaN,Visitante


## Exportando dados

In [702]:
base_jogos_tratada.columns

Index(['Link para o jogo', 'Time mandante (sigla)', 'Time mandante (nome)',
       'Time visitante (sigla)', 'Time visitante (nome)', 'Resultado',
       'Fase do torneio', 'Ida ou volta', 'Torneio', 'Data', 'Local',
       'Data e Hora', 'Ano', 'Mês', 'Dia', 'Hora', 'Minuto', '# jogo',
       'Time mandante (UF)', 'Time mandante (nome - UF)',
       'Time visitante (UF)', 'Time visitante (nome - UF)', 'Estádio',
       'Cidade', 'UF', 'Houve penaltis?', 'Gols mandante', 'Gols visitante',
       'Resultado (sem penaltis)', 'Penaltis mandante', 'Penaltis visitante',
       'Resultado dos penaltis', 'Resultado jogo'],
      dtype='object')

In [703]:
base_jogos_tratada.to_excel('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Base Jogos Tratada.xlsx')
# base_jogos_tratada.to_excel("Tabela Brasileirão.xlsx")

# Base "duplicada", para ver todos os jogos do time

In [704]:
base_jogos_tratada_duplicada = base_jogos_tratada.copy()
base_jogos_tratada_duplicada_temp = base_jogos_tratada.copy()

base_jogos_tratada_duplicada["Time (sigla)"] = base_jogos_tratada_duplicada["Time mandante (sigla)"]
base_jogos_tratada_duplicada["Time (nome)"] = base_jogos_tratada_duplicada["Time mandante (nome)"]

base_jogos_tratada_duplicada_temp["Time (sigla)"] = base_jogos_tratada_duplicada["Time visitante (sigla)"]
base_jogos_tratada_duplicada_temp["Time (nome)"] = base_jogos_tratada_duplicada["Time visitante (nome)"]

base_jogos_tratada_duplicada = pd.concat([base_jogos_tratada_duplicada, base_jogos_tratada_duplicada_temp])

# base_jogos_tratada_duplicada
base_jogos_tratada_duplicada[
    (base_jogos_tratada_duplicada["Time (nome)"] == "Palmeiras") \
        # (base_jogos_tratada_duplicada["Time (nome)"] == "Flamengo") \
    * (base_jogos_tratada_duplicada["Resultado"] != "Jogo ainda não ocorreu") \
    ].sort_values("Data e Hora", ascending = False).head(10)

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,...,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo,Time (sigla),Time (nome)
85,https://www.academiadasapostasbrasil.com/stats...,NaN,Palmeiras,NaN,Bolívar,4-0,Grupos - Rodada 6 - Grupo C,Volta,Libertadores,29/06/2023 21:00h,...,False,4.0,0.0,4 x 0,NaN,NaN,NaN,Mandante,NaN,Palmeiras
115,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,BOT,Botafogo,0 x 1,Rodada 12,Única,Brasileirão,"Dom, 25/06/2023 16:00\r\n ...",...,False,0.0,1.0,0 x 1,NaN,NaN,NaN,Visitante,PAL,Palmeiras
104,https://www.cbf.com.br/futebol-brasileiro/comp...,BAH,Bahia,PAL,Palmeiras,1 x 0,Rodada 11,Única,Brasileirão,"Qua, 21/06/2023 21:30\r\n ...",...,False,1.0,0.0,1 x 0,NaN,NaN,NaN,Mandante,PAL,Palmeiras
96,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Paulo,PAL,Palmeiras,0 x 2,Rodada 10,Única,Brasileirão,"Dom, 11/06/2023 16:00\r\n ...",...,False,0.0,2.0,0 x 2,NaN,NaN,NaN,Visitante,PAL,Palmeiras
69,https://www.academiadasapostasbrasil.com/stats...,NaN,Palmeiras,NaN,Barcelona,4-2,Grupos - Rodada 5 - Grupo C,Volta,Libertadores,07/06/2023 21:30h,...,False,4.0,2.0,4 x 2,NaN,NaN,NaN,Mandante,NaN,Palmeiras
88,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,COR,Coritiba,3 x 1,Rodada 09,Única,Brasileirão,"Dom, 04/06/2023 18:30\r\n ...",...,False,3.0,1.0,3 x 1,NaN,NaN,NaN,Mandante,PAL,Palmeiras
466,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza,PAL,Palmeiras,1 x 0,Oitavas de Final,Volta,Copa do Brasil,"Qua, 31/05/2023 19:00\r\n -...",...,False,1.0,0.0,1 x 0,NaN,NaN,NaN,Mandante,PAL,Palmeiras
77,https://www.cbf.com.br/futebol-brasileiro/comp...,ATL,Atlético Mineiro,PAL,Palmeiras,1 x 1,Rodada 08,Única,Brasileirão,"Dom, 28/05/2023 18:30\r\n ...",...,False,1.0,1.0,1 x 1,NaN,NaN,NaN,Empate,PAL,Palmeiras
53,https://www.academiadasapostasbrasil.com/stats...,NaN,Cerro Porteño,NaN,Palmeiras,0-3,Grupos - Rodada 4 - Grupo C,Volta,Libertadores,24/05/2023 19:00h,...,False,0.0,3.0,0 x 3,NaN,NaN,NaN,Visitante,NaN,Palmeiras
66,https://www.cbf.com.br/futebol-brasileiro/comp...,SAN,Santos,PAL,Palmeiras,0 x 0,Rodada 07,Única,Brasileirão,"Sáb, 20/05/2023 21:00\r\n ...",...,False,0.0,0.0,0 x 0,NaN,NaN,NaN,Empate,PAL,Palmeiras


In [705]:
base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado jogo"] == "Mandante") * \
    (base_jogos_tratada_duplicada["Time mandante (nome)"] == base_jogos_tratada_duplicada["Time (nome)"]),
    "Resultado do Time"] = "Vitória"
base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado jogo"] == "Visitante") * \
    (base_jogos_tratada_duplicada["Time visitante (nome)"] == base_jogos_tratada_duplicada["Time (nome)"]),
    "Resultado do Time"] = "Vitória"

base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado jogo"] == "Empate"),
    "Resultado do Time"] = "Empate"

base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado do Time"].isna() \
    * (base_jogos_tratada_duplicada["Resultado"] != "Jogo ainda não ocorreu")),
    "Resultado do Time"] = "Derrota"

# base_jogos_tratada_duplicada
base_jogos_tratada_duplicada[
    (base_jogos_tratada_duplicada["Time (nome)"] == "São Paulo") \
    # (base_jogos_tratada_duplicada["Time (nome)"] == "Palmeiras") \
    # (base_jogos_tratada_duplicada["Time (nome)"] == "Flamengo") \
    * (base_jogos_tratada_duplicada["Resultado"] != "Jogo ainda não ocorreu") \
    ].sort_values("Data e Hora", ascending = False).head(10)

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,...,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo,Time (sigla),Time (nome),Resultado do Time
183,https://www.academiadasapostasbrasil.com/stats...,NaN,São Paulo,NaN,Tigre,2-0,Grupos - Rodada 6 - Grupo D,Volta,Sul-Americana,27/06/2023 21:30h,...,2.0,0.0,2 x 0,NaN,NaN,NaN,Mandante,NaN,São Paulo,Vitória
113,https://www.cbf.com.br/futebol-brasileiro/comp...,CRU,Cruzeiro Saf,SAO,São Paulo,1 x 0,Rodada 12,Única,Brasileirão,"Sáb, 24/06/2023 21:00\r\n ...",...,1.0,0.0,1 x 0,NaN,NaN,NaN,Mandante,SAO,São Paulo,Derrota
100,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Paulo,ATH,Athletico Paranaense,2 x 1,Rodada 11,Única,Brasileirão,"Qua, 21/06/2023 19:00\r\n ...",...,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante,SAO,São Paulo,Vitória
96,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Paulo,PAL,Palmeiras,0 x 2,Rodada 10,Única,Brasileirão,"Dom, 11/06/2023 16:00\r\n ...",...,0.0,2.0,0 x 2,NaN,NaN,NaN,Visitante,SAO,São Paulo,Derrota
167,https://www.academiadasapostasbrasil.com/stats...,NaN,São Paulo,NaN,Dep. Tolima,5-0,Grupos - Rodada 5 - Grupo D,Volta,Sul-Americana,08/06/2023 19:00h,...,5.0,0.0,5 x 0,NaN,NaN,NaN,Mandante,NaN,São Paulo,Vitória
85,https://www.cbf.com.br/futebol-brasileiro/comp...,GRE,Grêmio,SAO,São Paulo,2 x 1,Rodada 09,Única,Brasileirão,"Dom, 04/06/2023 16:00\r\n ...",...,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante,SAO,São Paulo,Derrota
468,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Paulo,SPO,Sport,(5)1 x 3(3),Oitavas de Final,Volta,Copa do Brasil,"Qui, 01/06/2023 19:30\r\n -...",...,1.0,3.0,1 x 3,5.0,3.0,5 x 3,Visitante,SAO,São Paulo,Derrota
74,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Paulo,GOI,Goiás,2 x 1,Rodada 08,Única,Brasileirão,"Sáb, 27/05/2023 21:00\r\n ...",...,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante,SAO,São Paulo,Vitória
150,https://www.academiadasapostasbrasil.com/stats...,NaN,Puerto Cabello,NaN,São Paulo,0-2,Grupos - Rodada 4 - Grupo D,Volta,Sul-Americana,23/05/2023 21:30h,...,0.0,2.0,0 x 2,NaN,NaN,NaN,Visitante,NaN,São Paulo,Vitória
62,https://www.cbf.com.br/futebol-brasileiro/comp...,SAO,São Paulo,VAS,Vasco da Gama S.a.f.,4 x 2,Rodada 07,Única,Brasileirão,"Sáb, 20/05/2023 18:30\r\n ...",...,4.0,2.0,4 x 2,NaN,NaN,NaN,Mandante,SAO,São Paulo,Vitória


In [706]:
base_jogos_tratada_duplicada = base_jogos_tratada_duplicada.sort_index().reset_index().rename(columns = {"index": "index base tratada"})
base_jogos_tratada_duplicada

,index base tratada,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,...,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo,Time (sigla),Time (nome),Resultado do Time
0,0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf,FLU,Fluminense,0 x 3,Rodada 01,Única,Brasileirão,...,0.0,3.0,0 x 3,NaN,NaN,NaN,Visitante,AME,América Fc Saf,Derrota
1,0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf,FLU,Fluminense,0 x 3,Rodada 01,Única,Brasileirão,...,0.0,3.0,0 x 3,NaN,NaN,NaN,Visitante,FLU,Fluminense,Vitória
2,0,https://www.academiadasapostasbrasil.com/stats...,NaN,Aucas,NaN,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,...,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante,NaN,Flamengo,Derrota
3,0,https://www.academiadasapostasbrasil.com/stats...,NaN,Aucas,NaN,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,...,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante,NaN,Aucas,Vitória
4,1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,CUI,Cuiabá Saf,2 x 1,Rodada 01,Única,Brasileirão,...,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante,CUI,Cuiabá Saf,Derrota
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,477,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,SAO,São Paulo,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,...,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN,SAO,São Paulo,NaN
1340,478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians,AME,América Fc Saf,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,...,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN,AME,América Fc Saf,NaN
1341,478,https://www.cbf.com.br/futebol-brasileiro/comp...,COR,Corinthians,AME,América Fc Saf,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,...,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN,COR,Corinthians,NaN
1342,479,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense,FLA,Flamengo,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,...,NaN,NaN,Jogo ainda não ocorreu,NaN,NaN,NaN,NaN,FLA,Flamengo,NaN


## Exportando dados

In [707]:
base_jogos_tratada_duplicada.to_excel('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Base Jogos Tratada Duplicada.xlsx')
# base_jogos_tratada_duplicada.to_excel("Tabela Brasileirão Duplicada.xlsx")

# Tabela de pontos Brasileirão

In [708]:
tabela_brasileirao = base_jogos_tratada[(base_jogos_tratada["Torneio"] == "Brasileirão") * (base_jogos_tratada["Resultado"] != "Jogo ainda não ocorreu")].copy()

tabela_brasileirao

,Link para o jogo,Time mandante (sigla),Time mandante (nome),Time visitante (sigla),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data,...,Cidade,UF,Houve penaltis?,Gols mandante,Gols visitante,Resultado (sem penaltis),Penaltis mandante,Penaltis visitante,Resultado dos penaltis,Resultado jogo
0,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf,FLU,Fluminense,0 x 3,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",...,Belo Horizonte,MG,False,0.0,3.0,0 x 3,NaN,NaN,NaN,Visitante
1,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,CUI,Cuiabá Saf,2 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 16:00\r\n ...",...,Sao Paulo,SP,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante
2,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino,BAH,Bahia,2 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Braganca Paulista,SP,False,2.0,1.0,2 x 1,NaN,NaN,NaN,Mandante
3,https://www.cbf.com.br/futebol-brasileiro/comp...,ATH,Athletico Paranaense,GOI,Goiás,2 x 0,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Curitiba,PR,False,2.0,0.0,2 x 0,NaN,NaN,NaN,Mandante
4,https://www.cbf.com.br/futebol-brasileiro/comp...,FOR,Fortaleza,INT,Internacional,1 x 1,Rodada 01,Única,Brasileirão,"Sáb, 15/04/2023 18:30\r\n ...",...,Fortaleza,CE,False,1.0,1.0,1 x 1,NaN,NaN,NaN,Empate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,https://www.cbf.com.br/futebol-brasileiro/comp...,PAL,Palmeiras,BOT,Botafogo,0 x 1,Rodada 12,Única,Brasileirão,"Dom, 25/06/2023 16:00\r\n ...",...,Sao Paulo,SP,False,0.0,1.0,0 x 1,NaN,NaN,NaN,Visitante
116,https://www.cbf.com.br/futebol-brasileiro/comp...,AME,América Fc Saf,INT,Internacional,1 x 2,Rodada 12,Única,Brasileirão,"Dom, 25/06/2023 18:30\r\n ...",...,Belo Horizonte,MG,False,1.0,2.0,1 x 2,NaN,NaN,NaN,Visitante
117,https://www.cbf.com.br/futebol-brasileiro/comp...,RED,Red Bull Bragantino,GOI,Goiás,2 x 0,Rodada 12,Única,Brasileirão,"Dom, 25/06/2023 18:30\r\n ...",...,Braganca Paulista,SP,False,2.0,0.0,2 x 0,NaN,NaN,NaN,Mandante
118,https://www.cbf.com.br/futebol-brasileiro/comp...,SAN,Santos,FLA,Flamengo,2 x 3,Rodada 12,Única,Brasileirão,"Dom, 25/06/2023 18:30\r\n ...",...,Santos,SP,False,2.0,3.0,2 x 3,NaN,NaN,NaN,Visitante


In [709]:
# Estatísticas como mandante
tabela_brasileirao_mandante_temp = tabela_brasileirao.groupby(["Time mandante (nome)", "Resultado jogo"]).count()[["Link para o jogo"]]
tabela_brasileirao_mandante_temp = tabela_brasileirao_mandante_temp.join(
        tabela_brasileirao[["Time mandante (nome)", "Resultado jogo", "Gols mandante", "Gols visitante"]].groupby(["Time mandante (nome)", "Resultado jogo"]).sum()[["Gols mandante", "Gols visitante"]]
    ).reset_index()
tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Empate", "Pontos"] = \
    1 * tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Empate", "Link para o jogo"]
tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Mandante", "Pontos"] = \
    3 * tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Mandante", "Link para o jogo"]
tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Visitante", "Pontos"] = 0

tabela_brasileirao_mandante_temp = tabela_brasileirao_mandante_temp.rename(
    {"Link para o jogo": "Qtd jogos (como mandante)",
     "Time mandante (nome)": "Time",
     "Pontos": "Pontos (como mandante)",
     "Gols mandante": "Gols pró (como mandante)",
     "Gols visitante": "Gols contra (como mandante)",}, axis = 1)

tabela_brasileirao_mandante_temp = tabela_brasileirao_mandante_temp.set_index(["Time", "Resultado jogo"])
tabela_brasileirao_mandante_temp

# Estatísticas como visitante
tabela_brasileirao_visitante_temp = tabela_brasileirao.groupby(["Time visitante (nome)", "Resultado jogo"]).count()[["Link para o jogo"]]
tabela_brasileirao_visitante_temp = tabela_brasileirao_visitante_temp.join(
        tabela_brasileirao[["Time visitante (nome)", "Resultado jogo", "Gols mandante", "Gols visitante"]].groupby(["Time visitante (nome)", "Resultado jogo"]).sum()[["Gols mandante", "Gols visitante"]]
    ).reset_index()
tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Empate", "Pontos"] = \
    1 * tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Empate", "Link para o jogo"]
tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Visitante", "Pontos"] = \
    3 * tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Visitante", "Link para o jogo"]
tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Mandante", "Pontos"] = 0

tabela_brasileirao_visitante_temp = tabela_brasileirao_visitante_temp.rename(
    {"Link para o jogo": "Qtd jogos (como visitante)",
     "Time visitante (nome)": "Time",
     "Pontos": "Pontos (como visitante)",
     "Gols visitante": "Gols pró (como visitante)",
     "Gols mandante": "Gols contra (como visitante)",}, axis = 1)

tabela_brasileirao_visitante_temp = tabela_brasileirao_visitante_temp.set_index(["Time", "Resultado jogo"])
tabela_brasileirao_visitante_temp


# Totais
tabela_brasileirao_temp = tabela_brasileirao_visitante_temp.join(tabela_brasileirao_mandante_temp, how = "outer").fillna(0).reset_index()

# Pontos totais
tabela_brasileirao_temp["Pontos (total)"] = tabela_brasileirao_temp["Pontos (como visitante)"] + tabela_brasileirao_temp["Pontos (como mandante)"]

# Jogos totais
tabela_brasileirao_temp["Qtd jogos (total)"] = tabela_brasileirao_temp["Qtd jogos (como visitante)"] + tabela_brasileirao_temp["Qtd jogos (como mandante)"]

# Empates como mandante, como visitante e totais
tabela_brasileirao_temp["Qtd empates (como mandante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Empate", "Qtd jogos (como mandante)"]
tabela_brasileirao_temp["Qtd empates (como visitante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Empate", "Qtd jogos (como visitante)"]
tabela_brasileirao_temp = tabela_brasileirao_temp.fillna(0)
tabela_brasileirao_temp["Qtd empates (total)"] = tabela_brasileirao_temp["Qtd empates (como mandante)"] + tabela_brasileirao_temp["Qtd empates (como visitante)"]


# Derrotas totais
tabela_brasileirao_temp["Qtd derrotas (como visitante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Mandante", "Qtd jogos (como visitante)"]
tabela_brasileirao_temp["Qtd derrotas (como mandante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Visitante", "Qtd jogos (como mandante)"]
tabela_brasileirao_temp = tabela_brasileirao_temp.fillna(0)
tabela_brasileirao_temp["Qtd derrotas (total)"] = tabela_brasileirao_temp["Qtd derrotas (como mandante)"] + tabela_brasileirao_temp["Qtd derrotas (como visitante)"]

# Vitórias totais
tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Mandante", "Qtd vitórias (total)"] = tabela_brasileirao_temp["Qtd jogos (como mandante)"]
tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Visitante", "Qtd vitórias (total)"] = tabela_brasileirao_temp["Qtd jogos (como visitante)"]
tabela_brasileirao_temp = tabela_brasileirao_temp.fillna(0)

# Gols totais
tabela_brasileirao_temp["Gols pró"] = tabela_brasileirao_temp["Gols pró (como visitante)"] + tabela_brasileirao_temp["Gols pró (como mandante)"]
tabela_brasileirao_temp["Gols contra"] = tabela_brasileirao_temp["Gols contra (como visitante)"] + tabela_brasileirao_temp["Gols contra (como mandante)"]

# Tabela final
tabela_brasileirao_final = tabela_brasileirao_temp.groupby("Time").sum() \
    [["Pontos (total)", "Qtd jogos (total)", "Qtd vitórias (total)", "Qtd empates (total)", "Qtd derrotas (total)", "Gols pró", "Gols contra"]]
tabela_brasileirao_final["Saldo de gols"] = tabela_brasileirao_final["Gols pró"] - tabela_brasileirao_final["Gols contra"]
tabela_brasileirao_final["% aproveitamento"] = tabela_brasileirao_final["Pontos (total)"]/(tabela_brasileirao_final["Qtd jogos (total)"]*3)
tabela_brasileirao_final = tabela_brasileirao_final.sort_values(by = ["Pontos (total)", "Qtd vitórias (total)", "Saldo de gols", "Gols pró"], ascending = False)
tabela_brasileirao_final = tabela_brasileirao_final.reset_index().reset_index()
tabela_brasileirao_final["Posição"] = tabela_brasileirao_final["index"] + 1
tabela_brasileirao_final = tabela_brasileirao_final.set_index("Posição").drop("index", axis = 1)

tabela_brasileirao_final

<ipython-input-709-b2a45aeef56b>:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tabela_brasileirao_final = tabela_brasileirao_temp.groupby("Time").sum() \


,Time,Pontos (total),Qtd jogos (total),Qtd vitórias (total),Qtd empates (total),Qtd derrotas (total),Gols pró,Gols contra,Saldo de gols,% aproveitamento
Posição,,,,,,,,,,
1,Botafogo,30.0,12.0,10.0,0.0,2.0,20.0,7.0,13.0,0.833333
2,Grêmio,23.0,12.0,7.0,2.0,3.0,22.0,17.0,5.0,0.638889
3,Flamengo,22.0,12.0,7.0,1.0,4.0,24.0,17.0,7.0,0.611111
4,Palmeiras,22.0,12.0,6.0,4.0,2.0,22.0,10.0,12.0,0.611111
5,Fluminense,21.0,12.0,6.0,3.0,3.0,19.0,13.0,6.0,0.583333
6,Internacional,20.0,12.0,6.0,2.0,4.0,13.0,14.0,-1.0,0.555556
7,Red Bull Bragantino,20.0,12.0,5.0,5.0,2.0,21.0,14.0,7.0,0.555556
8,Fortaleza,20.0,12.0,5.0,5.0,2.0,15.0,9.0,6.0,0.555556
9,Athletico Paranaense,19.0,12.0,6.0,1.0,5.0,16.0,15.0,1.0,0.527778


In [710]:
# tabela_brasileirao_final.to_excel("Tabela Brasileirão.xlsx")
tabela_brasileirao_final.to_excel('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Tabela Brasileirão.xlsx')

# Análise exploratória

# Preparação de dados

## One hot encoding

## Balanceamento

## Separação treino e teste

## Normalização

# Classificação/Regressão

# TO-DO